In [2]:
from datasets import (
    get_dataset_config_names,
    concatenate_datasets,
    Dataset,
    load_dataset,
    Image,
)
import PIL
import io
from huggingface_hub import login
import os
import pandas as pd


all_configs = get_dataset_config_names("CharlyR/varbench-evaluation")

all_datasets: list[Dataset] = []

for config in all_configs:
    conf_ds = load_dataset("CharlyR/varbench-evaluation", config, split="tikz")
    config_name_column = [config] * len(conf_ds)
    all_datasets.append(conf_ds.add_column("config", config_name_column))


concat_datasets = concatenate_datasets(
    all_datasets
)  # works because all the metrics are the same, might eventually need to adapt it when more metrics are computed


concat_df: pd.DataFrame = concat_datasets.to_pandas()

concat_df = concat_df.explode(
    [
        col_name
        for col_name in concat_df.columns
        if "Metric" in col_name and not "best" in col_name
    ]
    + ["images_result", "predictions", "predictions_patches"]
)

concat_df = concat_df.dropna()

concat_datasets: Dataset = Dataset.from_pandas(concat_df)


concat_datasets = (
    concat_datasets.cast_column("image_solution", Image(decode=True))
    .cast_column("images_result", Image(decode=True))
    .cast_column("image_input", Image(decode=True))
)


concat_datasets.push_to_hub("CharlyR/varbench-metric-evaluation", config_name="raw")

/home/creux/miniconda3/envs/metricbench/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  1.44it/s]
No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/datasets/CharlyR/varbench-metric-evaluation/commit/20e733ba0b4aefbccf36c4fc60277b260f44ca4a', commit_message='Upload dataset', commit_description='', oid='20e733ba0b4aefbccf36c4fc60277b260f44ca4a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/CharlyR/varbench-metric-evaluation', endpoint='https://huggingface.co', repo_type='dataset', repo_id='CharlyR/varbench-metric-evaluation'), pr_revision=None, pr_num=None)

In [30]:
#WARNING:Removes the existing one
# Create an empty treated dataset (or filter existing data)
treated_df = concat_df.iloc[:0]  # Keeps the structure but removes rows

# Convert to Hugging Face Dataset
treated_dataset = Dataset.from_pandas(treated_df)
treated_dataset.push_to_hub("CharlyR/varbench-metric-evaluation", config_name="treated")

Creating parquet from Arrow format: 0ba [00:00, ?ba/s]:00<?, ?it/s]
Uploading the dataset shards: 100%|██████████| 1/1 [00:00<00:00,  3.55it/s]


CommitInfo(commit_url='https://huggingface.co/datasets/CharlyR/varbench-metric-evaluation/commit/41fb1ee2cb35934390a37bcf0887994f6051e678', commit_message='Upload dataset', commit_description='', oid='41fb1ee2cb35934390a37bcf0887994f6051e678', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/CharlyR/varbench-metric-evaluation', endpoint='https://huggingface.co', repo_type='dataset', repo_id='CharlyR/varbench-metric-evaluation'), pr_revision=None, pr_num=None)

### DEBUG

In [28]:
print(all_datasets[1]["config"][0])  # 1 and 4
df = all_datasets[1].to_pandas()
explod_col = [
    col_name
    for col_name in df.columns
    if "Metric" in col_name and not "best" in col_name
] + ["images_result", "predictions", "predictions_patches"]

df.explode(
    explod_col
)

simpleLLM_benchmark_deepseekr1distillllama70b_pk_5_t_1.5


,id,code,instruction,result_description,difficulty,patch,code_solution,image_solution,image_input,original_predictions,...,ChrfMetric,MSSSIMMetric,BleuMetric,TERMetric,PSNRMetric,ClipImageMetric,ChrfPatchMetric,CrystalBleuPatchMetric,CrystalBleuMetric,config
0,donkey_higher_mane,"\documentclass[tikz,border=5]{standalone}\n\us...",Make the mane of the donkey go upward a little...,A donkey with a high mane over its head,hard,"@@ -61,2 +61,2 @@\n- to [bend left] ++( 16, ...","\documentclass[tikz,border=5]{standalone}\n\us...",{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,"[\nAlright, so I need to help the user modify ...",...,98.500275,97.148544,95.006859,94.047623,46.219494,99.736816,73.293968,34.854549,92.739296,simpleLLM_benchmark_deepseekr1distillllama70b_...
0,donkey_higher_mane,"\documentclass[tikz,border=5]{standalone}\n\us...",Make the mane of the donkey go upward a little...,A donkey with a high mane over its head,hard,"@@ -61,2 +61,2 @@\n- to [bend left] ++( 16, ...","\documentclass[tikz,border=5]{standalone}\n\us...",{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,"[\nAlright, so I need to help the user modify ...",...,98.974251,98.461563,97.467621,97.832817,48.019768,99.881607,82.917809,62.712006,96.540497,simpleLLM_benchmark_deepseekr1distillllama70b_...
0,donkey_higher_mane,"\documentclass[tikz,border=5]{standalone}\n\us...",Make the mane of the donkey go upward a little...,A donkey with a high mane over its head,hard,"@@ -61,2 +61,2 @@\n- to [bend left] ++( 16, ...","\documentclass[tikz,border=5]{standalone}\n\us...",{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,"[\nAlright, so I need to help the user modify ...",...,99.067474,98.678612,97.467621,98.136642,49.100727,99.904991,36.491524,14.43965,96.584953,simpleLLM_benchmark_deepseekr1distillllama70b_...
0,donkey_higher_mane,"\documentclass[tikz,border=5]{standalone}\n\us...",Make the mane of the donkey go upward a little...,A donkey with a high mane over its head,hard,"@@ -61,2 +61,2 @@\n- to [bend left] ++( 16, ...","\documentclass[tikz,border=5]{standalone}\n\us...",{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,"[\nAlright, so I need to help the user modify ...",...,98.849823,95.36795,96.722496,95.180725,44.707695,98.082855,79.785591,49.461758,95.679916,simpleLLM_benchmark_deepseekr1distillllama70b_...
0,donkey_higher_mane,"\documentclass[tikz,border=5]{standalone}\n\us...",Make the mane of the donkey go upward a little...,A donkey with a high mane over its head,hard,"@@ -61,2 +61,2 @@\n- to [bend left] ++( 16, ...","\documentclass[tikz,border=5]{standalone}\n\us...",{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,"[\nAlright, so I need to help the user modify ...",...,98.500275,97.148544,95.006859,94.047623,46.219494,99.736816,73.293968,34.854549,92.739296,simpleLLM_benchmark_deepseekr1distillllama70b_...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49,cat_with_brown_pupils,"\documentclass[tikz,border=5]{standalone}\n\us...",Add brown pupils to the cat,A cat with brown pupils,medium,"@@ -38,0 +39 @@\n+ \fill [Brown] (0, 0) circl...","\documentclass[tikz,border=5]{standalone}\n\us...",{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,"[\n\nTo add brown pupils, I inserted fill comm...",...,95.967567,NaN,91.322723,82.65683,NaN,NaN,18.117064,0.506236,90.797691,simpleLLM_benchmark_deepseekr1distillllama70b_...
49,cat_with_brown_pupils,"\documentclass[tikz,border=5]{standalone}\n\us...",Add brown pupils to the cat,A cat with brown pupils,medium,"@@ -38,0 +39 @@\n+ \fill [Brown] (0, 0) circl...","\documentclass[tikz,border=5]{standalone}\n\us...",{'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...,{'bytes': b'\x89PNG\r

In [27]:
for i in range(len(df)):
    current_len=-1
    current_col = ""
    for col in explod_col:
        if current_len!=-1 and current_len !=len(df.iloc[i][col]):
            print(df.iloc[i])
            print(col)
            print(current_col)
            break
        current_col = col
        current_len = len(df.iloc[i][col])
        

id                                             shark_other_set_pectoral_fin
code                      \documentclass[tikz,border=5]{standalone}\n\us...
instruction               Add another set of pectoral fin to the shark, ...
result_description                   A shark with two sets of pectoral fins
difficulty                                                             hard
patch                     @@ -16,0 +17,6 @@\n+\fill [BlueGrey700] (11,10...
code_solution             \documentclass[tikz,border=5]{standalone}\n\us...
image_solution            {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...
image_input               {'bytes': b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHD...
original_predictions      [\n\nOkay, let's see. The user wants to add an...
predictions               [\documentclass[tikz,border=5]{standalone}\n\u...
image_result_indexes                                                     []
images_result                                            [None, None, None]
parsing_scor

In [12]:
concat_df[concat_df["id"] == "donkey_higher_mane"]["config"]


0      simpleLLM_benchmark_deepseekr1distillllama70b_...
50      simpleLLM_benchmark_llama3.18binstant_pk_1_t_0.7
100    simpleLLM_benchmark_llama3.370bversatile_pk_1_...
150         simpleLLM_benchmark_llama370b8192_pk_1_t_0.7
200      simpleLLM_benchmark_mixtral8x7b32768_pk_1_t_0.7
Name: config, dtype: object

In [3]:
len(concat_datasets)

581